예수께서 이르시되 내가 곧 길이요 진리요 생명이니 나로 말미암지 않고는 아버지께로 올 자가 없느니라 (요14:6)

-------
<center><img src="https://github.com/idebtor/DSpy/blob/6b3676c3de29f85d2e7f30679676d1fd96d88b17/images/WelcomeToDataStructures.jpg?raw=true" width=1000></center>

__NOTE:__ The following materials have been compiled and adapted from the numerous sources including my own. Please help me to keep this tutorial up-to-date by reporting any issues or questions. Send any comments or criticisms to `idebtor@gmail.com` Your assistances and comments will be appreciated.

--------


<b style="font-size:30px"> Chapter 5-4 Empirical Analysis of Sorting </b>

1. Review - Basic Sorting Algorithms
1. Emperical Analysis of Sorting
1. Exercises 

<b style="font-size:16px"> Notice: Reloading module </b>

Empirical Analysis에서는 file-based 방식으로 개발과 테스팅을 진행합니다. 이 때 유의할 것은 파일(모듈)을 import하여 사용한 후에, 모듈을 수정하고 다시 같은 모듈을 import하면, 그 전에 import한 모듈이 아직도 메모리에 cache되어 있습니다. 다시 import를 해도 기존의 module를 사용하게 됩니다. 

이러한 경우, `Kernel Restart`를 하여 다시 모듈을 강제적으로 시작할 수 있습니다. 

다음과 같은 매직 명령어를 미리 실행할 수도 있다고 합니다. (개인적으로 아래 명령어가 작동하지 않는 것을 많이 경험했습니다) 
```
%load_ext autoreload
%autoreload 2
```

In [2]:
%reload_ext autoreload
%autoreload 2

# Review - Sorting Algorithms
정렬은 한 컬렉션의 원소들을 일종의 순서대로 배치하는 것입니다. 예를 들어 리스트에서 문자의 길이나 알파벳 순으로 정렬할 수 있을 것이고, 도시 리스트는 인구, 지역 또는 우편 번호별로 정렬될 수 있을 것입니다. 이전 챕터에서 보았듯이(이진 검색과 같이), 원소들을 정렬하는 것은 연산량을 줄이거나 효과적인 탐색을 가능하게하는 등의 알고리즘을 만들 수 있게 됩니다.

## 정렬 알고리즘을 모듈로 만들고 사용하기

지금까지 정렬 알고리즘들을 모듈로 저장하십시오. 그리고, 아래 Bubble sort처럼 실행할 수 있는지 테스트하십시오. 아래는 n개의 원소를 무작위로 만든 배열(리스트)를 만들고 정렬하는 코드입니다. 

__Sample Run:__ 
```
import random
import bubble 

n = 10
a = [random.randint(10, 50) for i in range(n)]
print(a)
bubble.sort(a)
print(a)
```
__Expected Output:__
```
[50, 31, 36, 20, 21, 17, 12, 28, 18, 37]
[12, 17, 18, 20, 21, 28, 31, 36, 37, 50]
```

In [ ]:
%%writefile bubble.py
# 코드 1
def sort(a):
    for passn in range(len(a)-1,0,-1):
        for i in range(passn):
            if a[i]>a[i+1]:
                a[i], a[i+1] = a[i+1], a[i]

if __name__ == '__main__':
    a = [54,26,93,17,77,31,44,55,20]
    sort(a)
    print(a)

## 여러 정렬 알고리즘 실행하기 

정렬 알고리즘들을 리스트에 넣고, for-loop를 사용하여 실행하여 아래와 같이 출력하십시오. 

- 함수의 모듈과 이름은 함수의 `__module__`과 `__name__`에 저장되어 있습니다. 

__Sample Run:__   
```
n = 10
for f in [bubble.sort, selection.sort, insertion.sort, insertionYais.sort]:
    # your code here
```
__Expected Output:__    
```
bubble.sort
[23, 18, 15, 35, 25, 35, 37, 37, 31, 29]
[15, 18, 23, 25, 29, 31, 35, 35, 37, 37]
selection.sort
[42, 41, 10, 17, 40, 17, 33, 37, 23, 19]
[10, 17, 17, 19, 23, 33, 37, 40, 41, 42]
insertion.sort
[31, 32, 27, 18, 21, 44, 17, 12, 22, 49]
[12, 17, 18, 21, 22, 27, 31, 32, 44, 49]
insertionYais.sort
[27, 43, 22, 25, 18, 34, 48, 30, 43, 21]
[18, 21, 22, 25, 27, 30, 34, 43, 43, 48]
```

In [ ]:
import random
import bubble
import selection
import insertion
import insertionYais

n = 10
for f in [bubble.sort, selection.sort, insertion.sort, insertionYais.sort]:
    # your code here

# Empirical Analysis of Sorting

다음 프로그램(`timesort.py`)은 정렬 알고리즘(함수)들을 실행하여 시간을 측정하여 시간 복잡도를 테스트 할 수 있도록 돕습니다. 


## timesort.py

`timesort.py` 모듈에는 세 함수가 정의되어 있습니다. 

1. `timeTrials()`는 주어진 문제의 크기 `n`(problem size 즉 정렬할 요소의 수)에 대하여 정렬 함수를 실행합니다.  
1. `doublingTest()`는 문제의 크기 `n/2`와 `n`에 대한 실행 시간의 비율을 계산합니다.  예를 들면, `insertionYais`정렬일 경우에는 문제의 크기 즉 정렬할 요소가 두 배로 증가할 때, 시간의 비율은 약 4배로 증가하므로 시간 복잡도는 $O(n^2)$인 것을 확인할 할 수 있습니다. 이러한 실험을 통해, n이 증가함에 따라, 실행 시간이 급진적으로 증가하는 것을 관찰할 수 있으며, `insertionYais` 정렬과 같은 시간 복잡도를 가진 함수들이 문제의 크기가 큰 경우에는 사용할 수 없다는 것을 어렵지 않게 판단할 수 있습니다. 
1. `repeatTest`는 정렬 함수와 문제의 크기를 입력받아 반복적으로 `timeit.timeit`을 호출하여 실행 시간을 측정하여 반환합니다. 좀 더 정확한 실행 시간을 알 수 있습니다. 예를 들면, `repeatTest(f, n, trials=5, number=10)`는 정렬 n 개의 요소를 가진 무작위 배열을 생성하여 함수 `f`로 호출합니다. `number=10`번의 함수를 반복 실행하여 걸린 시간을 `trials=5`번 측정하여, 그 값들 중에 가장 낮은 trials의 값을 반환합니다. 

__Note:__   `timesort.py` 모듈을 실행하기 위해서는 `stopwatch` 모듈을 설치해야 합니다. 

1. __jupyter-lab의 code cell__ 에서 설치할 경우, 아래와 같이 시도해 볼 수 있습니다.   
```
!pip install stopwatch.py
```
1. __jupyter-lab의 code cell__ 에서 위의 느낌표 명령어가 작동되지 않을 경우 (Windows의 경우), 셀매직 명령어로 cmd창을 부르고, pip을 아래와 같이 시도해보십시오. Mac에서는 %%cmd대신에 %%bash를 사용해보길 바랍니다.
```
%%cmd
pip install stopwatch.py
```
1. Windows/MacOS/Linux Console에서 설치할 경우, 아래와 같이 시도해 보십시오.   
```
pip install stopwatch.py
```

In [ ]:
%%writefile timesort.py
from stopwatch import Stopwatch
from timeit import repeat
import random 

def timeTrials(func, n, trials):
    ''' Run func for an array a of n random[0..1), performing the 
        experiment trials times. Return the minimum wall-clock time it 
        took to run function.'''
    total = 0.0
    for t in range(trials):
        a = [random.uniform(0, 1) for i in range(n)]
        watch = Stopwatch()
        func(a)
        total += watch.duration
    return total

def doublingTest(func, n, trials, max_n = 8192):
    '''Perform a doubling test of the performance of function func starting
       at n, doubling n until n reaches max_n or over and writing the ratio of 
       the time for the current n and the time for the previous n each time 
       through the loop. Perform trials for each n.'''

    print(f"{func.__module__}.{func.__name__}()")
    print('%10s %6s %10s' % ('n', 'ratio', 'elapsed'))
    while n <= max_n:
        prev = timeTrials(func, n // 2, trials)
        curr = timeTrials(func, n,      trials)
        ratio = curr / prev
        print('%10d %6.2f %10.3f' % (n, ratio, curr))
        n *= 2
        
def repeatTest(f, n, trials=5, number=10):
    # generate an array of `n` items consisting of random integer 
    # values between 0 and n
    a = [random.randint(0, n) for i in range(n)]
    
    # set up the context and prepare the call to the specified f using the array. 
    # Only import the sort function if it's not the built-in `sorted()`.
    setup_code = f"from __main__ import {f}" if f != "sorted" else ""
    stmt = f"{f}.sort({a})"

    # execute timeit() 'trials'(default = 5) different times and 
    # return the time in seconds that each 'number' executions took
    # repeat: set number of timeit() calls, number: set timeit()'s number 
    times = repeat(setup=setup_code, stmt=stmt, repeat=trials, number=number)
    
    # display the funtion name, n and the minimu time it took 
    print(f"{f:>20s} {len(a):>7d} {min(times):10.4f}")

## 정렬 알고리즘의 Time Complexity 확인하기

아래의 __Sample Run__ 은 `insertionYais` 정렬함수를 실행하며, 문제의 크기 즉 정렬할 배열의 크기는 128부터 시작하여 매번 배가합니다. 문제의 크기가 n//2 인 경우와  n인 경우를 매번 10회를 실행하여 그 시간들을 비교하여 ratio를 구하고, elapsed는 n인 경우에 걸린 시간을 모두 합한 시간입니다.

__Sample Run:__  
```
doublingTest(insertionYais, 128, 10, 2048)

insertionYais.sort()
         n  ratio    elapsed
       128   3.78      0.008
       256   5.07      0.042
       512   3.54      0.174
      1024   3.40      0.626
      2048   4.23      2.600
```

In [ ]:
import timesort
import insertionYais

timesort.doublingTest(insertionYais.sort, 128, 50, 2048)

In [ ]:
import timesort
import insertion

timesort.repeatTest("insertion", 1024)

# Problem Based Learning - Emperical Analysis 

## insertion.sort() vs. insertionYais.sort()
우리는 insertion정렬을 구현한 두 개의 프로그램(`insertion.sort(), insertionYais.sort()`)을 경험하였습니다. 두 프로그램을 __Sample Run:__ 과 테스트했을 때, 시간 복잡도는 $O(n^2)$으로 수렴하지만, 두 프로그램의 실제적인 경과시간에는 현격한 차이가 있습니다. 



- 각자의 컴퓨터에서 `doublingTest()`를 실행하고, 두 프로그램을 비교 분석하고 어느 부분에서 차이가 나는지, 그러한 코드 차이가 왜 시간 차이를 만들어내는지 그 근본적인 이유를 설명하십시오.  

__Sample Run:__

```
import timesort
import insertion
import insertionYais

for f in [insertion.sort, insertionYais.sort]:
    timesort.doublingTest(f, 512, 10, 4096)
```

```
insertion.sort()
         n  ratio    elapsed
       512   4.13      0.110
      1024   4.31      0.463
      2048   4.50      1.989
      4096   4.16      8.271
insertionYais.sort()
         n  ratio    elapsed
       512   6.18      0.161
      1024   4.77      0.754
      2048   3.96      2.695
      4096   4.04     10.772
```

__Your Analysis here:__



## [Challenge] Test and optimize Insertion sort (bonus +0.5p)

#### - If you find a better code that has been well optimized for insertion sort, you can get points by importing it with a clear source notation. Of course, developing it yourself is also welcome.  

1. Develop a new version of Insertion sort and name it `insertion2.sort()`.
1. Write an emperical test code showing the following output. 

__Expected Output:___
```
insertion.sort()
         n  ratio    elapsed
       512   5.04      0.127
      1024   4.10      0.489
      2048   4.24      2.003
insertionYais.sort()
         n  ratio    elapsed
       512   4.16      0.179
      1024   3.80      0.582
      2048   3.71      2.557
insertion2.sort()
         n  ratio    elapsed
       512   3.84      0.090
      1024   3.69      0.335
      2048   3.99      1.567
```


__My Solution:__

In [ ]:
#%%writefile insertion2.py
def sort(a):
    for i in range(1, len(a)):  # start from 1 since 1st element is trivially sorted
        pass
       
if __name__ == '__main__':
    a = [54, 26, 93, 17, 77, 31, 44, 55, 20]
    print("before:", a)
    insertion2.sort(a)
    print(" after:", a)

In [3]:
# script here to test insertion, insertionYais and insertion2 sorting algorithms
import timesort
import insertion
import insertionYais
import insertion2


insertion.sort()
         n  ratio    elapsed
       512   4.22      0.113
      1024   3.45      0.467
      2048   3.91      1.964
insertionYais.sort()
         n  ratio    elapsed
       512   3.26      0.159
      1024   3.65      0.632
      2048   3.71      2.608
insertion2.sort()
         n  ratio    elapsed
       512   4.51      0.078
      1024   4.38      0.410
      2048   4.18      1.593


## Selection sort vs. Insertion sort

간단한 정렬 알고리즘의 시간복잡도는 대개 $O(n^2)$이지만, Selection sort가 거의 항상 Bubble sort 보다 더 빠르게 정렬합니다. 그러나, Selection sort가 Insertion sort 보다 항상 빠른 것은 아닙니다.  Selection sort는 비교 횟수는 많지만, Bubble sort에 비해 실제로 교환(swap)하는 횟수는 적기 때문에 많은 교환이 일어나야 하는 자료상태에서 비교적 효율적입니다.

Insertion Sort는 2번째 원소부터 시작하여 그 앞(왼쪽)의 원소들과 비교하여 삽입할 위치를 지정한 후, 원소를 뒤로 옮기고(shift) 지정된 자리에 자료를 삽입 하여 정렬하는 알고리즘입니다. 최선의 경우 O(N)이라는 빠른 효율성을 가지고 있고, __문제의 크기가 작은 경우에 대해__, Quick sort 혹은 Merge sort와 같은 정렬 알고리즘의 일부로 사용될 만큼 좋은 정렬 알고리즘입니다.

Selection sort와 Insertion sort는 k번째 반복 이후, 첫번째 k 요소가 정렬된 순서로 온다는 점에서 유사하다. 하지만, Selection sort는 k+1번째 요소를 찾기 위해 나머지 모든 요소들을 탐색하지만 Insertion sort는 k+1번째 요소를 배치하는 데 필요한 만큼의 요소만 탐색하기 때문에 더 효율적으로 실행된다는 차이가 있습니다.

__Task:__   

1. Empirically, find the upper bound `n` of "small arrays" size that insertion sort usually is faster than selection sort, quick sort, or merge sort. 
2. Write the timing test script that shows insertion sort faster than others for "small arrays".


__Solution:__  


In [ ]:
import timesort


# References
- Problem Solving with Algorithms and Data Structures using Python, [Here](https://runestone.academy/runestone/books/published/pythonds3/Introduction/WhyStudyDataStructuresandAbstractDataTypes.html)
    - Chapter 6.6 Sorting [Here](https://runestone.academy/runestone/books/published/pythonds3/SortSearch/sorting.html)

------
예수께서 이르시되 내가 곧 길이요 진리요 생명이니 나로 말미암지 않고는 아버지께로 올 자가 없느니라 (요14:6)